<a href="https://colab.research.google.com/github/bcury/Machine-Learning-Projects/blob/main/Car's_Predict_LiinearRegression_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Car's predict

In [2]:
pip install jovian

     |████████████████████████████████| 68 kB 4.3 MB/s 
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6501 sha256=354cc97590569312f2c744e16adb0a73f0977fb12d644120efffb9979b9849ce
  Stored in directory: /root/.cache/pip/wheels/2a/ea/87/dd57f1ecb4f0752f3e1dbf958ebf8b36d920d190425bcdc24d
Successfully built uuid


In [3]:
import torch
import jovian
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

In [4]:
Data_Frame = pd.read_csv("/content/car data.csv")

In [5]:
Data_Frame.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [6]:
Data_Frame.describe()

,Year,Selling_Price,Present_Price,Kms_Driven,Owner
count,301.000000,301.000000,301.000000,301.000000,301.000000
mean,2013.627907,4.661296,7.628472,36947.205980,0.043189
std,2.891554,5.082812,8.644115,38886.883882,0.247915
min,2003.000000,0.100000,0.320000,500.000000,0.000000
25%,2012.000000,0.900000,1.200000,15000.000000,0.000000
50%,2014.000000,3.600000,6.400000,32000.000000,0.000000
75%,2016.000000,6.000000,9.900000,48767.000000,0.000000
max,2018.000000,35.000000,92.600000,500000.000000,3.000000


In [7]:
Data_Frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


In [17]:
my_name = "Bruno Cury"
def customize_dataset(dataframe_raw, rand_str):
  Data = Data_Frame.copy(deep=True)
  Data = Data.sample(int(0.95*len(Data)), random_state=int(ord(rand_str[0])))
  Data.Year = Data.Year*ord(rand_str[1])/100.
  Data.Selling_Price = Data.Selling_Price*ord(rand_str[2])/100.
  if ord(rand_str[3]) % 2 == 1:
        Data = Data.drop(['Car_Name'], axis=1)
  return Data

Data = customize_dataset(Data_Frame, my_name)
Data.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
103,Bajaj Dominar 400,2299.38,1.6965,1.60,1200,Petrol,Individual,Manual,0
64,fortuner,2299.38,38.6100,36.23,6000,Diesel,Dealer,Automatic,0
35,sx4,2292.54,3.4515,7.74,49998,CNG,Dealer,Manual,0
280,brio,2297.10,6.1425,5.90,14465,Petrol,Dealer,Manual,0
256,city,2298.24,11.9925,13.60,49562,Petrol,Dealer,Manual,0


In [18]:
input_cols = ["Year","Present_Price","Kms_Driven","Owner"]
categorical_cols = ["Fuel_Type","Seller_Type","Transmission"]
output_cols = ["Selling_Price"]

In [19]:
def Data_to_arrays(Data):
    # Make a copy of the original dataframe
    Data1 = Data.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        Data1[col] = Data1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = Data1[input_cols].to_numpy()
    targets_array = Data1[output_cols].to_numpy()
    return inputs_array, targets_array

inputs_array, targets_array = Data_to_arrays(Data)
inputs_array, targets_array

(array([[2.29938e+03, 1.60000e+00, 1.20000e+03, 0.00000e+00],
        [2.29938e+03, 3.62300e+01, 6.00000e+03, 0.00000e+00],
        [2.29254e+03, 7.74000e+00, 4.99980e+04, 0.00000e+00],
        ...,
        [2.29482e+03, 5.10000e-01, 3.20000e+04, 0.00000e+00],
        [2.29596e+03, 7.49000e+00, 3.90000e+04, 0.00000e+00],
        [2.29710e+03, 5.80000e+00, 4.00230e+04, 0.00000e+00]]),
 array([[ 1.6965],
        [38.61  ],
        [ 3.4515],
        [ 6.1425],
        [11.9925],
        [ 5.4405],
        [ 0.7605],
        [ 5.733 ],
        [ 5.265 ],
        [ 6.786 ],
        [ 6.9615],
        [10.8225],
        [ 3.8025],
        [24.2775],
        [ 6.1425],
        [ 0.1989],
        [ 7.3125],
        [ 6.1425],
        [10.647 ],
        [ 1.3455],
        [ 3.6855],
        [ 9.6525],
        [ 0.351 ],
        [ 0.351 ],
        [10.8225],
        [ 9.0675],
        [ 2.2815],
        [ 3.2175],
        [ 0.468 ],
        [ 3.51  ],
        [ 1.2285],
        [ 0.5265],
     

In [20]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

dataset = TensorDataset(inputs, targets)
train_ds, val_ds = random_split(dataset, [228, 57])
batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

**Regression Linear Model using PyTorch **

In [25]:
input_size = len(input_cols)
output_size = len(output_cols)

class CarsModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, xb):
        out = self.linear(xb)                         
        return out

    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs) 

        loss = F.l1_loss(out, targets)                         
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))
            
model = CarsModel()

list(model.parameters())

[Parameter containing:
 tensor([[ 0.0848,  0.2007, -0.1781, -0.3519]], requires_grad=True),
 Parameter containing:
 tensor([-0.2769], requires_grad=True)]

Training Model

In [26]:
# Eval algorithm
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

# Fitting algorithm
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

# Check the initial value that val_loss have
result = evaluate(model, val_loader)
print(result)

{'val_loss': 7877.52490234375}


In [27]:
# Start with the Fitting
epochs = 90
lr = 1e-8
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 7236.5654
Epoch [40], val_loss: 6596.3408
Epoch [60], val_loss: 5956.4722
Epoch [80], val_loss: 5315.5903
Epoch [90], val_loss: 4996.0298


In [28]:
# Train repeatdly until have a 'good' val_loss
epochs = 20
lr = 1e-9
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 4932.0898


Lets use the model above to predict cars Prices

In [29]:
# Prediction Algorithm
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

# Testing the model with some samples
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([2.2971e+03, 1.4700e+00, 2.6000e+04, 0.0000e+00])
Target: tensor([1.2870])
Prediction: tensor([-2737.1597])


lets compare with what we got

In [30]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([2.2982e+03, 4.4300e+00, 1.2500e+04, 0.0000e+00])
Target: tensor([3.3930])
Prediction: tensor([-1209.2294])


its very similar, so it worked out